In [1]:
from importlib.metadata import version
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import math
from dataclasses import dataclass
from matplotlib import pyplot as plt
import time
import os

# Pour torch si vous avez un GPU
# device = "cpu" if not torch.cuda.is_available() else "cuda"
device = "cpu" # Pour forcer l'utilisation du CPU

In [2]:
# Environement potentielement testé
from environnement.environnement import Environnement as env # mother class
from environnement.environnement1 import Environnement1 as env1
from environnement.environnement2Str import Environnement2 as env2Str
from environnement.environnement3Str import Environnement3 as env3Str
from environnement.environnement6Str import Environnement6 as env6Str
from environnement.small_loop import small_loop
from environnement.small_loop import WorldDisplay
from ipywidgets import Output

# model machine learning
from model.DeepNN import *
from model.Tokenizer import *
from model.CustomDataSet import CustomDataSet
from outil import *
from inter.interactions import Interaction
from inter.simpleInteraction import simpleInteraction as inter

# L'agent qui devine les meilleurs patterns
Je vais maintenant implémenter un mécanisme qui va tenter de trouver les meilleurs pattern.
## Critere d'un pattern :
Un bon pattern est un pattern pour lequel Prob * Valance est élever. Nous voulons que l'agent sout capable de dire : cette action est bonne car après je vais pouvoir avoir une bonne valence. Donc il faut prédire sur du long terme.
## L'idée
L'idée est de regarder les probabilités que donne le modèl pour certainne action. Créer des enchainements d'actions, en découlera des prédictions de pattern. Nous pourrons alors éxecuter des bonnes séquence d'action.

## Valence
La valence est primordiale pour notre. C'est elle qui va faire emerger un comportement à notre agent. 

### Exemple small loop
Prennons un environement un peu complexe. On peut imaginer un monde en 2 dimensions, sur une grille de 5x5 avec les bords et la case du centre comme obstacle. Nous voulons que notre agent apprenne a ce balader sans se prendre d'obstacle. Graphiquement cela donne :

In [3]:
demo_env = small_loop(x=1, y=1, theta=0)
# demo_env.display_world()

Imaginons que nous avons un modèle prédictif parfait. Il faut choisir la meilleur action. Pour simplifier disons que nous pouvons prédire que sur 3 interactions futur, sans connaitre la position absolut du robot la meilleur solution est la suivante :
- **Feel front** regarder si devans nous il y a un obstacle. Nous voulons que la prédiction soit égal à 50% wall 50% Empty. En tout cas une probabilité qui s'en rapproche.
- **move** Nous voulons que le robot agisse en fonction, soit avancé soit touner.

Si nous somme dans une intéraction ou nous avons `feel front` alors nous voulons directement faire l'action qui correspond.

In [4]:
class AgentPE:
    def __init__(self, model, all_outcomes, all_actions, tokenizer, optimizer=None, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self._otimizer = optimizer
        self._loss_func = loss_func
        self._tokenizer:SimpleTokenizerV1 = tokenizer
        self._all_outcomes = all_outcomes
        self._all_actions = all_actions
        self._history_act = []
        self._history_fb = []


    def fit(self, actions:list, outcomes:list, nb_epoch:int= 5, validate_loader=None):
        """
        Fonction d'entrainement de l'agent 
        Avec data set custom, le model prends en inputs plusieurs données
        """
        context_lenght = self._model.input_size
        print(f'context_lenght {context_lenght}')
        if len(actions) + len(outcomes) < context_lenght:
            raise Exception("Not enough data to train model")

        actions = [self._tokenizer.encode(act) for act in actions]
        outcomes = self._tokenizer.encode(outcomes)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.train()
            # actions = torch.tensor(actions, dtype=torch.float).to(device) # On passe toutes les actions que l'agent a fais
            # outcomes = torch.tensor(outcomes, dtype=torch.long).to(device) # On passe toutes oputcomes qu'il a 

            data_loarder = CustomDataSet(actions=actions, outcomes=outcomes, # On va créer un dataset
                         context_lenght=context_lenght, dim_out=len(self._all_outcomes))
            
            # A la place de donnée x = [act1] y = [out1]
            # Nous voulons donné : x = [act1, out1, act2] y = [out2]

            data_loader = torch.utils.data.DataLoader( # On utilise torch pour charger les données
                data_loarder,batch_size=32, shuffle=True)

            train_with_batch(model=self._model, 
                    train_loader=data_loader,
                    optimizer=self._otimizer,
                    loss_func=self._loss_func,
                    nb_epochs=nb_epoch,
                    validate_loader=validate_loader,
                    print_=True)
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass

    def get_prediction(self, action):
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.nn.functional.softmax(x, dim=0)
        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return x
       # def check_all_actions(self):
    #     act_to_test = None
    #     for act in self._all_actions:
    #         if act not in self._history_act:
    #             act_to_test = act
    #             break
    #     return act_to_test
    
    # def decide(self):
    #     """
    #     Fonction qui choisit l'action a faire en fonction des prédictions \
    #     du modèles entrainné. Nous renforçons choisisons les actions que \
    #     ou le modèle n'est pas sûr.
    #     """

    #     act_test = self.check_all_actions()
    #     if act_test:
    #         print("i don't know", act_test)
    #         self._action = act_test
    #         return act_test

    #     best_act = self._all_actions[0]
    #     best_expected_val = -np.inf

    #     # On vérifie que l'on a vue assez d'interaction pour faire des prédictions
    #     if len(self._history_act) + len(self._history_fb) < self._model.input_size:
    #         print("Not enough data to make a decision")
    #         return best_act

    #     # Vérifie si le modèles est sur de sa prédiction
    #     for act in self._all_actions:
    #         probs:torch.Tensor = self.get_prediction(act)
    #         max_prob = torch.max(probs).item()
    #         # Formule utiliser : 1 / n + 0.5 / n
    #         print(f'for action {act} probs {probs} max_prob {max_prob}')
    #         if max_prob < 1 / probs.size(dim=0) + 0.5 / probs.size(dim=0):
    #             print("je ne suis pas sur de ", act)
    #             if len(self._all_actions) + len(self._all_outcomes) > self._model.input_size:
    #                 self.fit(self._history_act, self._history_fb, validate_loader=None)

    #         probs:torch.Tensor = self.get_prediction(act)
    #         if max_prob < 1 / probs.size(dim=0) + 0.5 / probs.size(dim=0):
    #             print("je n'arrive pas à être sur de ", act)
    #             return act
            
    #         # Si le modèle as une prédiction sur, on regarde sa valance
    #         predi = self._tokenizer.decode(torch.argmax(probs, dim=0).item())
    #         expected_val = self._valance[inter(act, predi)]
    #         if expected_val > best_expected_val:
    #             best_act = act
    #             best_expected_val = expected_val
    #             print(f"Action: {act}, Expected valance: {expected_val}")
    #     self._action = best_act
    #     return best_act

    # Modifier
    def predict(self, action):
        """
        Funciton de prédiction
        """

        if len(self._history_act) + len(self._history_fb) + 1 < self._model.input_size:
            raise Exception("Not enough data to train model")
        
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.argmax(x, dim=0).item()

        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return self._tokenizer.decode(x)

    # Modofier
    def action(self, outcome, fit=True, validate_loader=None):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            self._history_fb.append(outcome)
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, " 
                  f"\033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m")
            if self._predicted_outcome != outcome:
                if len(self._history_act) + len(self._history_fb) > self._model.input_size:
                    self.fit(self._history_act, self._history_fb, validate_loader=validate_loader)
                else:
                    self._action = self._all_actions[0]

            # Maintenant nous choisissons la prochaine action en fonction de la valance
            self._action = str(np.random.choice(self._all_actions))
            if len(self._history_act) + len(self._history_fb) + 1 > self._model.input_size:
                self._predicted_outcome = self.predict(self._action)
            self._predicted_outcome = self.predict(self._action)
            self._history_act.append(self._action)
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)            
            print(f"Action de base : {self._action} Prediction: {self._predicted_outcome}")
        
        return self._action, self._predicted_outcome

In [5]:
env_test2 = small_loop(x=1, y=1, theta=0)

model_ML = DeepNetwork(hidden_size=[16], input_size=3, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=0.01, weight_decay=0.001)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

agent_test2 = AgentPE(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(200):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")
    

liste hidden init [16]
======================= iteration 0 =======================
Action de base : forward Prediction: None
action forward predi None outcome wall
Action choisie : forward 0 


======================= iteration 1 =======================
Action: forward, Prediction: None, Outcome: wall,  Satisfaction: False 
action forward predi wall outcome wall
Action choisie : forward 0 


======================= iteration 2 =======================
Action: forward, Prediction: wall, Outcome: wall,  Satisfaction: True 
action forward predi wall outcome wall
Action choisie : forward 0 


======================= iteration 3 =======================
Action: forward, Prediction: wall, Outcome: wall,  Satisfaction: True 
action forward predi wall outcome wall
Action choisie : forward 0 


======================= iteration 4 =======================
Action: forward, Prediction: wall, Outcome: wall,  Satisfaction: True 
action feel_front predi wall outcome wall
Action choisie : feel_front 0 



In [6]:
# On teste certaines sequences de prédiction
print("Test de prédiction")
seq = tokenizer.encode(['forward', 'wall', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'forward', 'wall', 'forward', probabilité {prob}, decode {deocde}")

seq = tokenizer.encode(['feel_front', 'empty', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
print(prob)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'feel_front', 'empty', 'forward' probabilité {prob} decode {deocde}")

seq = tokenizer.encode(['feel_front', 'wall', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'feel_front', 'wall', 'forward', probabilité {prob} decode {deocde}")

# Cas non prédictible correctement :
seq = tokenizer.encode(['forward', 'empty', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence forward', 'empty', 'forward' :  probabilité {prob}, decode {deocde}")

seq = tokenizer.encode(['turn_left', 'empty', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'turn_left', 'empty', 'forward'  probabilité {prob}, decode {deocde}")

seq = tokenizer.encode(['turn_right', 'empty', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'turn_right', 'empty', 'forward', probabilité {prob}, decode {deocde}")

Test de prédiction
Prédiction de la séquence 'forward', 'wall', 'forward', probabilité tensor([0.5397, 0.4603], grad_fn=<SoftmaxBackward0>), decode wall
tensor([0.2282, 0.7718], grad_fn=<SoftmaxBackward0>)
Prédiction de la séquence 'feel_front', 'empty', 'forward' probabilité tensor([0.2282, 0.7718], grad_fn=<SoftmaxBackward0>) decode empty
Prédiction de la séquence 'feel_front', 'wall', 'forward', probabilité tensor([0.8081, 0.1919], grad_fn=<SoftmaxBackward0>) decode wall
Prédiction de la séquence forward', 'empty', 'forward' :  probabilité tensor([0.1690, 0.8310], grad_fn=<SoftmaxBackward0>), decode empty
Prédiction de la séquence 'turn_left', 'empty', 'forward'  probabilité tensor([0.1418, 0.8582], grad_fn=<SoftmaxBackward0>), decode empty
Prédiction de la séquence 'turn_right', 'empty', 'forward', probabilité tensor([0.1593, 0.8407], grad_fn=<SoftmaxBackward0>), decode empty


In [7]:
# 1er version de la fonction qui détermine l'expected valence :

def tempo_recursif_expective_valance(model:nn.Module, env, seq:list,
                                    max_depth:int, valance:dict, tokenizer:SimpleTokenizerV1,
                                    seuil:float=0.2, proba:float = 1,
                                    seq_predi:list = []):
    """
    """
    if max_depth == 0:
        return {}
    max_depth -= 1

    if proba < seuil:
        return {}
    
    model.eval()
    exceptive_valance = {}
    for act in env.get_actions():
        new_seq = seq_predi + [act]
        seq_to_predict = seq + [tokenizer.encode(act)]
        seq_to_predict = torch.tensor(seq_to_predict, dtype=torch.float).to(device)
        x = model(seq_to_predict)
        # Transforme x into list proba
        probs = torch.nn.functional.softmax(x, dim=0).tolist()
        # for each outcomes we want proba with act
        for i, out in enumerate(env.get_outcomes()):
            tmp_new_seq = new_seq + [out]
            tmp_proba = probs[i] * proba
            if tmp_proba < seuil:
                continue
            tempo = float(np.round(valance[inter(act, out)] * tmp_proba, decimals=4))
            # input(f'seq {seq_predi} act {act} out {out} proba {tmp_proba} valance {valance[(act, out)]} tempo {tempo}')

            exceptive_valance.update(
                tempo_recursif_expective_valance(model=model, env=env, seq=seq[2:] + [tokenizer.encode(act), tokenizer.encode(out)],
                                            max_depth=max_depth, valance=valance, seuil=seuil, 
                                            proba=tmp_proba, seq_predi=tmp_new_seq.copy(), tokenizer=tokenizer)
            )
            exceptive_valance[str(tmp_new_seq)] = tempo
    return exceptive_valance


valence = {
    inter('forward', 'empty') : 10,
    inter('forward', 'wall') : -30,
    inter('turn_left', 'empty') : -3,
    inter('turn_left', 'wall') : -3,
    inter('turn_right', 'empty') : -3,
    inter('turn_right', 'wall') : -3,
    inter('feel_front', 'wall') : -5,
    inter('feel_front', 'empty') : -5,
}

print(env_test2.get_actions())
print(env_test2.get_outcomes())


res = tempo_recursif_expective_valance(model=model_ML, env=env_test2,
        seq=tokenizer.encode(['feel_front', 'wall']), max_depth=3, valance=valence, tokenizer=tokenizer, seuil=0.2)

print(res)

# Top 5 of sequences with the best expected valance
top_5 = sorted(res.items(), key=lambda x: x[1], reverse=True)[:5]
for top in top_5:
    print(f"Sequence: {top[0]} Expected valance: {top[1]}")


['forward', 'turn_left', 'turn_right', 'feel_front']
['wall', 'empty']
{"['forward', 'wall', 'forward', 'wall', 'forward', 'wall']": -7.0612, "['forward', 'wall', 'forward', 'wall', 'forward', 'empty']": 2.0075, "['forward', 'wall', 'forward', 'wall', 'turn_left', 'empty']": -0.7845, "['forward', 'wall', 'forward', 'wall', 'turn_right', 'wall']": -0.725, "['forward', 'wall', 'forward', 'wall', 'feel_front', 'wall']": -2.0625, "['forward', 'wall', 'forward', 'wall']": -13.0835, "['forward', 'wall', 'forward', 'empty', 'forward', 'empty']": 3.0909, "['forward', 'wall', 'forward', 'empty', 'turn_left', 'empty']": -0.9924, "['forward', 'wall', 'forward', 'empty', 'turn_right', 'empty']": -0.9368, "['forward', 'wall', 'forward', 'empty', 'feel_front', 'wall']": -1.3573, "['forward', 'wall', 'forward', 'empty']": 3.7196, "['forward', 'wall', 'turn_left', 'wall', 'turn_right', 'empty']": -0.6274, "['forward', 'wall', 'turn_left', 'wall']": -0.9706, "['forward', 'wall', 'turn_left', 'empty', '

In [8]:
print("probalité")
seq = tokenizer.encode(['forward', 'wall', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'turn_right', 'empty', 'forward', probabilité {prob}, decode {deocde}")

probalité
Prédiction de la séquence 'turn_right', 'empty', 'forward', probabilité tensor([0.5397, 0.4603], grad_fn=<SoftmaxBackward0>), decode wall


In [9]:
class AgentPEBis:
    def __init__(self, model, all_outcomes, all_actions, tokenizer, valence:dict,
                optimizer=None, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self._otimizer = optimizer
        self._loss_func = loss_func
        self._tokenizer:SimpleTokenizerV1 = tokenizer
        self._all_outcomes = all_outcomes
        self._all_actions = all_actions
        self._history_act = []
        self._history_fb = []
        self._valence = valence


    def fit(self, actions:list, outcomes:list, nb_epoch:int= 5, validate_loader=None):
        """
        Fonction d'entrainement de l'agent 
        Avec data set custom, le model prends en inputs plusieurs données
        """
        context_lenght = self._model.input_size
        print(f'context_lenght {context_lenght}')
        if len(actions) + len(outcomes) < context_lenght:
            raise Exception("Not enough data to train model")

        actions = [self._tokenizer.encode(act) for act in actions]
        outcomes = self._tokenizer.encode(outcomes)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.train()
            data_loarder = CustomDataSet(actions=actions, outcomes=outcomes,
                         context_lenght=context_lenght, dim_out=len(self._all_outcomes))

            data_loader = torch.utils.data.DataLoader(
                data_loarder,batch_size=32, shuffle=True)

            train_with_batch(model=self._model, 
                    train_loader=data_loader,
                    optimizer=self._otimizer,
                    loss_func=self._loss_func,
                    nb_epochs=nb_epoch,
                    validate_loader=validate_loader,
                    print_=True)
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass

    def get_prediction(self, action):
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.nn.functional.softmax(x, dim=0)
        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return x

    def predict(self, action):
        """
        Funciton de prédiction
        """

        if len(self._history_act) + len(self._history_fb) + 1 < self._model.input_size:
            raise Exception("Not enough data to train model")
        
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.argmax(x, dim=0).item()

        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return self._tokenizer.decode(x)
    
    def recursif_expective_valance(self, seq:list, max_depth:int, seuil:float=0.2, proba:float = 1,
                                    seq_predi:list = []):
        """
        """
        if max_depth == 0:
            return {}
        max_depth -= 1

        if proba < seuil:
            return {}
        
        self._model.eval()
        exceptive_valance = {}
        for act in self._all_actions:
            new_seq = seq_predi + [act]
            seq_to_predict = seq + [tokenizer.encode(act)]
            seq_to_predict = torch.tensor(seq_to_predict, dtype=torch.float).to(device)
            x = self._model(seq_to_predict)
            # Transforme x into list proba
            probs = torch.nn.functional.softmax(x, dim=0).tolist()
            # for each outcomes we want proba with act
            for i, out in enumerate(self._all_outcomes):
                tmp_new_seq = new_seq + [out]
                tmp_proba = probs[i] * proba
                if tmp_proba < seuil:
                    continue
                tempo = float(np.round(self._valence[inter(act, out)] * tmp_proba, decimals=4))
                # input(f'seq {seq_predi} act {act} out {out} proba {tmp_proba} valance {valance[(act, out)]} tempo {tempo}')
                exceptive_valance.update(
                    self.recursif_expective_valance(seq=seq[2:] + [tokenizer.encode(act), tokenizer.encode(out)],
                                                max_depth=max_depth, seuil=seuil, 
                                                proba=tmp_proba, seq_predi=tmp_new_seq.copy())
                )
                exceptive_valance[str(tmp_new_seq)] = tempo
        return exceptive_valance
        

    def decide(self):
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        seq = self._tokenizer.encode(x)
        res = self.recursif_expective_valance(seq=seq, max_depth=3, seuil=0.2)
        top_5 = sorted(res.items(), key=lambda x: x[1], reverse=True)[:5]
        print(f"Top 5 of sequences with the best expected valance for {x}")
        for top in top_5:
            print(f"Sequence: {top[0]} Expected valance: {top[1]}")
        
        print(f"Action choisie : {eval(top_5[0][0])[0]}")
        return eval(top_5[0][0])[0]


    # Modifier
    def action(self, outcome, decide=True, validate_loader=None):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            self._history_fb.append(outcome)
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, " 
                  f"\033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m")
            if self._predicted_outcome != outcome:
                if len(self._history_act) + len(self._history_fb) > self._model.input_size:
                    self.fit(self._history_act, self._history_fb, validate_loader=validate_loader)
                else:
                    self._action = self._all_actions[0]

            # Maintenant nous choisissons la prochaine action en fonction de la valance
            if decide:
                self._action = self.decide()
            else :
                self._action = str(np.random.choice(self._all_actions))
            if len(self._history_act) + len(self._history_fb) + 1 > self._model.input_size:
                self._predicted_outcome = self.predict(self._action)
            self._predicted_outcome = self.predict(self._action)
            self._history_act.append(self._action)
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)            
            print(f"Action de base : {self._action} Prediction: {self._predicted_outcome}")
        
        return self._action, self._predicted_outcome

In [10]:
out = Output()
EnvDisplay = WorldDisplay()


In [16]:
env_test2 = small_loop(x=1, y=1, theta=0)

model_ML = DeepNetwork(hidden_size=[16], input_size=3, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=0.01, weight_decay=0.001)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

valence = {
    inter('forward', 'empty') : 1,
    inter('forward', 'wall') : -10,
    inter('turn_left', 'empty') : -3,
    inter('turn_left', 'wall') : -3,
    inter('turn_right', 'empty') : -3,
    inter('turn_right', 'wall') : -3,
    inter('feel_front', 'wall') : -5,
    inter('feel_front', 'empty') : -5,
}

agent_test2 = AgentPEBis(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    valence=valence,
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(10):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")

# On l'entraine 200 fois



for i in range(10):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, decide=True)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")
    # sleep(0.5)
    # EnvDisplay.show(env_test2.get_world(), env_test2.get_robot(), out)
    # display(out)
    # time.sleep(0.5)
    

liste hidden init [16]
======================= iteration 0 =======================
Action de base : forward Prediction: None
action forward predi None outcome wall
Action choisie : forward 0 


======================= iteration 1 =======================
Action: forward, Prediction: None, Outcome: wall,  Satisfaction: False 
action feel_front predi empty outcome wall
Action choisie : feel_front 0 


======================= iteration 2 =======================
Action: feel_front, Prediction: empty, Outcome: wall,  Satisfaction: False 
context_lenght 3
Epoch 1/5, Loss: 1.0000
Epoch 2/5, Loss: 0.6584
Epoch 3/5, Loss: 0.4122
Epoch 4/5, Loss: 0.2508
Epoch 5/5, Loss: 0.1520
action forward predi wall outcome wall
Action choisie : forward 0 


======================= iteration 3 =======================
Action: forward, Prediction: wall, Outcome: wall,  Satisfaction: True 
action turn_left predi wall outcome empty
Action choisie : turn_left 0 


======================= iteration 4 ===============

# Renouveau

Je reprends l'agent 1 et ces facultées a prédire dans un environement. Et je vais ajouter un mécanisme de décision. Ce mécanisme de décision est censé plustard réduire la complexité car va selectionner les bon début d'interactions et ne pas ajouter des actions inutil.

In [12]:
raise Exception("stop")

Exception: stop

In [ ]:
class Agent1WithPast:
    def __init__(self, model, all_outcomes, all_actions, tokenizer, optimizer=None, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self._otimizer = optimizer
        self._loss_func = loss_func
        self._tokenizer:SimpleTokenizerV1 = tokenizer
        self._all_outcomes = all_outcomes
        self._all_actions = all_actions
        self._history_act = []
        self._history_fb = []

    # Modifier
    def fit(self, actions:list, outcomes:list, nb_epoch:int= 5, validate_loader=None, threshold:float=None):
        """
        Fonction d'entrainement de l'agent
        """
        context_lenght = self._model.input_size
        if len(actions) + len(outcomes) < context_lenght:
            raise Exception("Not enough data to train model")

        # actions = [[self._tokenizer.encode(act)] for act in actions]
        # outcomes = self._tokenizer.encode(outcomes)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.train()
            # actions = torch.tensor(actions, dtype=torch.float).to(device) 
            # outcomes = torch.tensor(outcomes, dtype=torch.long).to(device) 
            data_set = CustomDataSet(actions=actions, outcomes=outcomes,
                                    dim_out=len(self._all_outcomes),
                                    tokenizer=self._tokenizer,context_lenght=context_lenght)
            
            # A la place de donnée x = [act1] y = [out1]
            # Nous voulons donné : x = [act1, out1, act2] y = [out2]

            data_loader = torch.utils.data.DataLoader( # On utilise torch pour charger les données
                data_set,batch_size=32, shuffle=True)


            # print("je dois fit avec :")
            # print(actions)
            # print(outcomes)
            train_with_batch(model=self._model, 
                    train_loader=data_loader,
                    optimizer=self._otimizer,
                    loss_func=self._loss_func,
                    nb_epochs=nb_epoch,
                    validate_loader=validate_loader,
                    threshold=threshold,
                    print_=True)
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass

    # Modifier
    def predict(self, action):
        """
        Funciton de prédiction
        """
        if len(self._history_act) + len(self._history_fb) + 1 < self._model.input_size:
            raise Exception("Not enough data to predic model")
        
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.argmax(x, dim=0).item()

        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return self._tokenizer.decode(x)

    # Modofier
    def action(self, outcome, fit=True, validate_loader=None):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            self._history_fb.append(str(outcome))
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, " 
                  f"\033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m")
            # Le model s'est trompé, on le réentraine
            if self._predicted_outcome != outcome:
                # Sauf si il n'a pas encore assez de donnée
                if len(self._history_act) + len(self._history_fb) > self._model.input_size:
                    self.fit(actions=self._history_act, 
                            outcomes=self._history_fb, 
                            validate_loader=validate_loader, 
                            nb_epoch=int(np.log(len(self._history_act))) * 30 + 5,
                            threshold=0.2)

            self._action = np.random.choice(self._all_actions)
            if len(self._history_act) + len(self._history_fb) + 1 > self._model.input_size:
                self._predicted_outcome = self.predict(self._action)
            self._history_act.append(str(self._action))
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)
            print(f'len all interaction {len(self._history_act) + len(self._history_fb)}')
            print(f'input size {self._model.input_size}')
            print(f"Action de base : {self._action} Prediction: {self._predicted_outcome}")
        
        return self._action, self._predicted_outcome

In [ ]:
input_dim = 21
env_test2 = small_loop(x=1, y=1, theta=0)
model_ML = DeepNetwork(hidden_size=[128, 32, 16], input_size=input_dim, output_size=2)
optimizer = torch.optim.SGD(model_ML.parameters(), lr=1e-5, weight_decay=1e-6)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(
    create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

agent_test2 = Agent1WithPast(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    optimizer=optimizer,
    loss_func=loss_func,
    tokenizer=tokenizer)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(500):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, validate_loader=None)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    # agent_test2._model._init_weights()
    # print(f'action {action} predi {predi} outcome {outcome}')
    # print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    # env_test2.display_world()
    # time.sleep(0.5)
    print("\n\n")

see_evolued_acc(pourcent_by_10[input_dim:])

data_set = CustomDataSet(actions=agent_test2._history_act[:-1], outcomes=agent_test2._history_fb, 
                        dim_out=len(env_test2.get_outcomes()),
                        tokenizer=tokenizer,
                        context_lenght=input_dim)

data_loader_visu = torch.utils.data.DataLoader(data_set, batch_size=32, shuffle=True)
no_good = find_no_good(model=model_ML, test_loader=data_loader_visu)

for x, label, pred in no_good:
    # x tensor to list
    x = x.tolist()
    label = label.tolist()
    pred = pred.tolist()
    print(f"Action: {tokenizer.decode(x)} Label: {tokenizer.decode(label)} Prediction: {pred}")

# Count in list agent_test2._history_fb
count = {}
for x in agent_test2._history_fb:
    if x in count:
        count[x] += 1
    else:
        count[x] = 1
print(count)
print(f'En moyenne le modèl a {sum(history_good[input_dim:]) / (len(history_good) + input_dim)} chance de prédire un outcome')

test time training
multi op memory literal
